In [81]:
# Imports
import math
import importlib
import ipywidgets as widgets
from IPython.display import display

In [105]:
adefaultvalue=0.8333
ddefaultvalue=10
wdefaultvalue=0.5
QTSdefaultvalue=0.3

a=widgets.FloatText(value=adefaultvalue, disabled=False)
print("Enter your single crochet height:")
display(a)

d=widgets.IntText(value=ddefaultvalue, disabled=False)
print("Enter your dot-to-dot length:")
display(d)


w=widgets.FloatSlider(value=wdefaultvalue,disabled=False,min=0,max=1)
print("How wide should a strand be, as a proportion of its dot-to-dot length?")
display(w)

QTS=widgets.FloatSlider(value=QTSdefaultvalue,disabled=False,min=0,max=1)
print("How long should the straight section on a quarter turn be, as a proportion of the dot-to-dot length?")
display(QTS)



#####################



button = widgets.Button(description="restore defaults")
#output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    a.value=adefaultvalue
    d.value=ddefaultvalue
    w.value=wdefaultvalue
    QTS.value=QTSdefaultvalue
#    with output:
#        print("Defaults restored")

button.on_click(on_button_clicked)

#w = widgets.IntSlider()
#display(w)

#v = widgets.IntSlider()
#display(v)

#t=widgets.Text(value='Hello World!', disabled=False)
#print("Enter text")
#display(t)


#out = widgets.Output(layout={'border': '1px solid black'})
#out

#with out:
#    for i in range(10):
#        print(i, 'Hello world!')



Enter your single crochet height:


FloatText(value=0.8333)

Enter your dot-to-dot length:


IntText(value=10)

How wide should a strand be, as a proportion of its dot-to-dot length?


FloatSlider(value=0.5, max=1.0)

How long should the straight section on a quarter turn be, as a proportion of the dot-to-dot length?


FloatSlider(value=0.3, max=1.0)

Button(description='restore defaults', style=ButtonStyle())

Output()

In [75]:
if a.value<0:
    print("height of a sc should be positive")
else:
    if a.value>1:
        print("Generally single crochets are shorter than they are wide.")
    elif a.value<.5:
        print("That is a very short stitch.")
    sch=a.value
    print("single crochet height:",a.value)

if d.value<3:
    print("that length is too short")
else:
    dtd=d.value
    print("dot-to-dot length:",dtd,"stitches")
    
QTStraight=QTS.value
widthinstitches=dtd*w.value
widthinrows=widthinstitches/sch
HalfRows=round(widthinrows/2)#needs to be an even number of rows
TotalRows=2*HalfRows
QTstraightsts=round(QTStraight*dtd)

print("number of rows:",TotalRows)



single crochet height: 0.8333
dot-to-dot length: 10 stitches
number of rows: 6


# Gauge and Proportions

In [9]:

#########################################
###fill these in###
#sch=20/24 ###defined by widget now
#dtd=10 ###defined by widget now
#widthinstitches=dtd*.5 ###defined by widget now
#QTStraight=.3 ###defined by widget now

#########################################
#sch: single crochet height 
    #20 stitches and 24 rows is a square
    #so each stitch has width 1 (by definition) and height 5/6 (by measurement)
    #4mm hook, blue acrylic, tight:
    #30 stitches and 34/35 rows is a square
#dtd:
    #dot-to-dot length: 10 sc default

#widthinstitches
    #default ratio: width of a strand is half the dtd length
    
#QTstraight:
    #for a quarter turn, what number of the dtd distance should be straight, before the turn
    #after playing around, 30% seems to look good.
    #For original version, set to 0

#########################################
#other constants calculated here
#widthinrows=widthinstitches/sch
#HalfRows=round(widthinrows/2)#needs to be an even number of rows
#TotalRows=2*HalfRows
#QTstraightsts=round(QTStraight*dtd)

#print("Dot-to-dot length:",dtd,"stitches")
#print("Number of rows:",TotalRows)
#print("Straight length in a quarter turn:",QTstraightsts,"stitches")

##check proportions
###############################################
#dtd vs width
#if widthinstitches>dtd:
#    print("*Warning*")
#    print(" The strands are too thick. Your strand will be wider than the distance between two intersections.")
###no longer allowed by slider
###############################################
#max distance from the centre of the strand is Delta
#basicall widthinstitches/2, but accounting for rounding:
Delta = HalfRows*sch

#quarter turn's straight segment's total length should not exceed u-(sqrt 2 - 1)Delta,
#where Delta is the max distance from the middle of the strand to the edge
maxs = dtd-2.414*Delta
if QTstraightsts > maxs:
    print("*Warning*")
    print(" A quarter turn might overrun its boundaries. Max length of the quarter straight section is",maxs)
    
#Delta shoudn't exceed dtd(sqrt 2 - 1), or else the points and twists overrun their feature walls
maxd = dtd*(2**.5-1)
if Delta > maxd:
    print("*Warning*")
    print("A point or twist might overrun its boundaries. To avoid, decrease widthinstitches")
    
#Delta shouldn't exceed dtd/sqrt 8, or else the J-type and C-type corners will overrun their feature walls
#Runs aren't a standard component, so for many knots, this warning doesn't apply
maxj = dtd/8**.5
if Delta > maxj:
    print("*Possible Warning*")
    print("If you plan to use a run with a J-type or C-type corner, it might overrun its boundaries. To avoid, decrease widthinstitches")
    print("If your knot does not have these components, disregard")

# Generate Strings
Each component has a string giving stitch measures. 

# Quarter Turn

Including foundation chain as Row 0, there are TotalRows+1 rows. The middle of these will have radius dtd. The radius changes by sch (single crochet height) each row.

Straight stitches are based on the QTStraight parameter; these are the same very row. Remaining stitches make a quarter turn, radius varies by row: dtd distance minus straight distance. 

In [10]:
QuarterTurn=[]

for r in range(-HalfRows,HalfRows+1):
    RadiusShift=sch*r
    Radius=dtd+RadiusShift-QTstraightsts
    UnroundedArcLength = math.pi/2*Radius
    ArcLength = round(UnroundedArcLength)
    QuarterTurn.append(ArcLength)
print(QuarterTurn)#

[7, 8, 10, 11, 12, 14, 15]


# Point

Each half of a point is an eighth-circle, then a straight line, meeting at a square corner. It goes:
arc, single crochets, 2 ch, single crochets, arc.

Arc: radius as in quarter turn.

Point: The middle round should have length $S=\left[u\left(1-\frac1{\sqrt 2}\right)\right]$ on each side. This increases/decreases by one each row, adjusted so that the innermost row has a positive measure.

In [11]:
PointArc=[]
PointCorner=[]

S=round(dtd*(1-(.5**.5)))
S=max(S,HalfRows+1)

for r in range(-HalfRows,HalfRows+1):
    #arc
    RadiusShift=sch*r
    Radius=dtd+RadiusShift
    UnroundedArcLength = math.pi/4*Radius
    ArcLength = round(UnroundedArcLength)
    PointArc.append(ArcLength)
    #corner
    Side=S+r
    PointCorner.append(Side)
print(PointArc)
print(PointCorner)



[6, 7, 7, 8, 9, 9, 10]
[1, 2, 3, 4, 5, 6, 7]


# Twist

One-eighth turn, followed by a half-turn with a smaller radius, then a final one-eighth turn. 

Inner radius: $\text{dtd}\left(1-\frac{1}{\sqrt2}\right)+\text{RadiusShift}$

For the half-circle, there is a danger of a very tight innermost row, meaning increases (2 sc in 1 st) and decrease (sc2tog) might not be enough. Will have to use other stitches (3 sc in 1 st, sc3tog) or edit the innermost row post hoc.

In [12]:
TwistTop=[]
for r in range(-HalfRows,HalfRows+1):
    #arcradius
    RadiusShift=sch*r
    Radius=dtd+RadiusShift
    #halfcircle radius
    HRadius = dtd*(1-math.sqrt(.5))+RadiusShift
    UnroundedArcLength = math.pi*HRadius
    ArcLength = round(UnroundedArcLength)
    TwistTop.append(max(ArcLength,1))
print(TwistTop)
for sts in range (1,len(TwistTop)):
    difference=TwistTop[sts]/TwistTop[sts-1]
    if difference>2:
        TwistTop[sts-1]=math.ceil(TwistTop[sts]/2)
print(TwistTop)


[1, 4, 7, 9, 12, 14, 17]
[2, 4, 7, 9, 12, 14, 17]


# Runs

A *single component* can consist of one or more runs. 

* There are four corner types, each of which can be inner or outer: D, J, L, C
  * D and J only occur at the start or end of a component.
  * L and C only occur between two runs, inside one component
* Between each corner type, count the number of feature walls that make up the run. That is, the combined number of feature walls on the two adjacent parallel lines of feature walls
* The function "run" defined below will return a string of instructions, one entry per row, for a single component. 
  * When generating the pattern for an entire strand, you can pass a string of instructions, one entry per row
  * You can have the function called as part of your pattern, e.g. pattern=["S",run("OD",5,"OD"),"S"]
  * Or, you can first generate strings, e.g. R=run("OD",5,"OD") pattern=["S",R,"S"]

The syntax for describing a run to plug into the run function is: alternating strings and ints. 
*  Strings: OD, ID, OJ, IJ, OL, IL, OC, IC
*  Ints: must be at least 2; otherwise it's not a run
*  Program will print an error message if there's a physical problem:
   * When an end / middle component is used in the middle / end, and
   * when the outer/inner designations don't match with the lengths



In [13]:
#content of this cell has been moved to generate_run.py
dtype=[]
#lenghts of arc. 
for r in range(-HalfRows,HalfRows+1):
    turn=int(math.pi/4*(dtd/0.8284+r*sch))
    dtype.append(turn)
print("D:",dtype)
#dtype also has straight bit 0.9142*dtd, same every row
daddstraight=0.2071*dtd

#jtype: lengths of arc. 1: quarter turn (next to straight section). 2: eighth turn (starts or ends run).
jtype1=[]
jtype2=[]


#jaddstraight: straight lengths adjacent to run:
#each row the same; don't round because it will be added to middle straight part
jaddstraight=dtd/2

#The j-type lists are made in two stages.
#For chunky dimensions, it's possible for one arclength to be negative and the other positive.
#In this case, add them together to have a total arclength as close as possible to the ideal length
for r in range(-HalfRows,HalfRows+1):
    turn1temp=math.pi/2*(dtd*0.2071+r*sch)
    turn2temp=math.pi/4*(dtd*0.8536+r*sch)
    if turn1temp<0:
            turn1=0
            turn2=max(int(turn1temp+turn2temp),0)
    elif turn2temp<0:
            turn2=0
            turn1=max(int(turn1temp+turn2temp),0)
    else:
        turn1=int(turn1temp)
        turn2=int(turn2temp)
    jtype1.append(turn1)
    jtype2.append(turn2)
    
print("J:",jtype1,jtype2)
print(jaddstraight)

#L-type
#straight: subtract 1, becuase ch / dec will be specified explicitly
#corner instructions the same for every row (2ch or 2 dec)
#straight instructions different for every row
#this is the only corner type with changing straights
laddstraight=[]
for r in range(-HalfRows,HalfRows+1):
    laddstraight.append(dtd/1.414+r*sch-1)
print("L str:",laddstraight)

#C-type
caddstraight=dtd/8**.5
ctype=[]
for r in range(-HalfRows,HalfRows+1):
    cturn=max(0,int(math.pi*(caddstraight+r*sch)))
    ctype.append(cturn)
print("C:",ctype)



D: [7, 8, 8, 9, 10, 10, 11]
J: [0, 0, 1, 3, 4, 5, 7] [4, 5, 6, 6, 7, 8, 8]
5.0
L str: [3.5722357850070727, 4.405535785007073, 5.238835785007073, 6.072135785007073, 6.905435785007073, 7.738735785007073, 8.572035785007074]
C: [3, 5, 8, 11, 13, 16, 18]


In [15]:
###
#input: a name, and a description of the run
#descriptions: list of alternating strings and ints
#strings: ID, OD, IJ, OJ, IL, OL, IC, OC
#function will return a string; entries are instructions for each row
#this way, can put run instructions inside pattern=[] string
def run(run_input):
    instructions=[]
    ##separate out corners and lengths
    corners=[]
    lengths=[]
    for i in range(len(run_input)):  
        if i%2==0:
            corners.append(run_input[i])
        else:
            lengths.append(run_input[i])

#    print(corners)
#    print(lengths)
    #now the corners and lengths are defined
    #check that corners start and end appropriately
    #doing this here rather than later because the later iteration doesn't make it to the last index
    if corners[0][1]!="D" and corners[0][1]!="J":
        print("Segment must start with D or J")
    if corners[len(corners)-1][1]!="D" and corners[len(corners)-1][1]!="J":
        print("Segment must end with D or J")
    for m in range(1,len(corners)-1):
        if corners[m][1]=="D" or corners[m][1]=="J":
            print("Runs are joined by C or L type corners, not D or J. Check join",m)

    for r in range(-HalfRows,HalfRows+1):
        row=r+HalfRows
        if row==0:
            stitch="\ch~"
        else:
            stitch="\sc~"
#        print("***** row ",row,"*****")
        delta=sch*r
        ###stitch height * row, where delta=0 in the centre of the strand
        instructions.append("")#add a blank string to start off the instructions for this row
        skipnextcorner=0 #indicator: did we just do a C? If so, the next corner has no instructions, so don't make a blank / / for it
##########################################################################################
        for m in range(len(lengths)): ##iterate over each run in the segment
##########################################################################################
            #the current run has corner1 end type, length feature walls in the run, and corner2 end type
            corner1=corners[m]
            orientation1=corner1[0]
            corner2=corners[m+1]
            orientation2=corner2[0]
            length=lengths[m]
    ############################################################################
            #Parity check: even-lenths change orientation, odd-lengths preserve orientation
            if length%2==0:
                if orientation1==orientation2:
                    print("Error: an even-length run has opposite orientations on each end  (one inner, one outer)")
            else:
                if orientation1!=orientation2:
                    print("Error: an odd-length run has the same orientations on each end  (both inner, or both outer)")
    #############################################################################
    #middle
            straightmiddle=(length-2)/1.414*dtd
    #first corner
    ############################################################################
            #straight1:  straight segment associated with first corner. Will be added to straightmiddlepart
            if corner1=="ID" or corner1=="OD":
                straight1=daddstraight
                if corner1=="ID":##inside to outside
                    turnlength=dtype[row]
                    if r!=-HalfRows:
                        difference=dtype[row]-dtype[row-1]
                        if difference==0:
                            change=""
                        else:
                            change="(+"+str(difference)+")"
                    else:
                        change=""

                else:##outside to inside
                    turnlength=dtype[TotalRows-row]
                    if r!=-HalfRows:
                        difference=dtype[TotalRows-row+1]-dtype[TotalRows-row]
                        if difference==0:
                            change=""
                        else:
                            change="(-"+str(difference)+")"
                    else:
                        change=""

                corner1instructions=str(turnlength)+" "+stitch+change

            elif corner1=="IJ" or corner1=="OJ":
                straight1=jaddstraight#since this is the *first* corner, it starts outside the run
                if corner1=="IJ": #inside to outside
                    if row!=0:
                        difference2=jtype2[row]-jtype2[row-1]
                        difference1=jtype1[row]-jtype1[row-1]
                        if difference2!=0:
                            change2=" (+"+str(difference2)+")"
                            if jtype2[row-1]==0:
                                change2=" (+"+str(difference2)+", new segment)"
                        else:
                            change2=""
                        if difference1!=0:
                            change1=" (+"+str(difference1)+")"
                            if jtype1[row-1]==0:
                                change1=" (+"+str(difference1)+", new segment)"
                        else:
                            change1=""
                    else:
                        change2=""
                        change1=""
                    if row%2==0:
                        corner1instructions=str(jtype2[row])+" "+stitch+change2+" / "+str(jtype1[row])+" "+stitch+change1
                    else:
                        corner1instructions=str(jtype1[row])+" "+stitch+change1+" / "+str(jtype2[row])+" "+stitch+change2
                else:#outside to inside
                    if row!=0:
                        difference2=jtype2[TotalRows-row+1]-jtype2[TotalRows-row]
                        difference1=jtype1[TotalRows-row+1]-jtype1[TotalRows-row]
                        if difference2!=0:
                            change2="(-"+str(difference2)+")"
                            if jtype2[TotalRows-row]==0:
                                change2="(-"+str(difference2)+", end segment)"
                        else:
                            change2=""
                        if difference1!=0:
                            change1="(-"+str(difference1)+")"
                            if jtype1[TotalRows-row]==0:
                                    change1="(-"+str(difference1)+", end segment)"
                        else:
                            change1=""
                    else:
                        change1=""
                        change2=""
                    if row%2==0:
                        corner1instructions=str(jtype2[TotalRows-row])+" "+stitch+change2+" / "+str(jtype1[TotalRows-row])+" "+stitch+change1
                    else:
                        corner1instructions=str(jtype1[TotalRows-row])+" "+stitch+change1+" / "+str(jtype2[TotalRows-row])+" "+stitch+change2
            elif corner1=="IL" or corner1=="OL":
                if corner1=="IL":
                    straight1=laddstraight[row]
                    corner1instructions="2 ch~"
                else:
                    straight1=laddstraight[TotalRows-row]
                    corner1instructions="2 dec~"
            elif corner1=="IC" or corner1=="OC":
                straight1=caddstraight
                if corner1=="IC":#inside to outside
                    if row!=0:
                        difference=ctype[row]-ctype[row-1]
                        if difference!=0:
                            change="(+"+str(difference)+")"
                        else:
                            change=""
                    else:
                        change=""
                    corner1instructions=str(ctype[row])+" "+stitch+change
                else: 
                    if row!=0:
                        difference=ctype[TotalRows-row+1]-ctype[TotalRows-row]
                        if difference!=0:
                            change="(-"+str(difference)+")"
                        else:
                            change=""
                    else:
                        change=""
                corner1instructions=str(ctype[TotalRows-row])+" "+stitch+change
            else:
                print("unknown corner1 type")

    #second corner
    ############################################################################
            #straight2:  straight segment associated with second corner. Will be added to straightmiddlepart        
            if corner2=="ID" or corner2=="OD":
                straight2=daddstraight
                if corner2=="ID":#inside to outside
                    if row!=0:
                        difference=dtype[row]-dtype[row-1]
                        if difference!=0:
                            change="(+"+str(difference)+")"
                        else:
                            change=""
                    else:
                        change=""
                    corner2instructions=str(dtype[row])+" "+stitch+" "+change
                else: #outside to inside
                    if row!=0:
                        difference=dtype[TotalRows-row+1]-dtype[TotalRows-row]
                        if difference!=0:
                            change="(-"+str(difference)+")"
                        else:
                            change=""
                    else:
                        change=""
                    corner2instructions=str(dtype[TotalRows-row])+" "+stitch+change
            elif corner2=="IJ" or corner2=="OJ":
                straight2=jaddstraight
                if corner2=="IJ":#in to out
                    if row!=0:
                        difference1=jtype1[row]-jtype1[row-1]
                        difference2=jtype2[row]-jtype2[row-1]
                        if difference1!=0:
                            change1=" (+"+str(difference1)+")"
                            if jtype1[row-1]==0:
                                change1=" (+"+str(difference1)+", new segment)"
                        else:
                            change1=""
                        if difference2!=0:
                            change2=" (+"+str(difference2)+")"
                            if jtype2[row-1]==0:
                                change2=" (+"+str(difference2)+", new segment)"
                        else:
                            change2=""
                    else:
                        change1=""
                        change2=""
                    if row%2==0:
                        corner2instructions=str(jtype1[row])+" "+stitch+change1+" / "+str(jtype2[row])+" "+stitch+change2
                    else:
                        corner2instructions=str(jtype2[row])+" "+stitch+change2+" / "+str(jtype1[row])+" "+stitch+change1
                else: #out to in
                    if row!=0:
                        difference1=jtype1[TotalRows-row+1]-jtype1[TotalRows-row]
                        difference2=jtype2[TotalRows-row+1]-jtype2[TotalRows-row]
                        if difference1!=0:
                            change1="(-"+str(difference1)+")"
                            if jtype1[TotalRows-row]==0:
                                change1="(-"+str(difference1)+", end segment)"
                        else:
                            change1=""
                        if difference2!=0:
                            change2="(-"+str(difference2)+")"
                            if jtype2[TotalRows-row]==0:
                                change2="(-"+str(difference2)+", end segment)"
                        else:
                            change2=""
                    else:
                        change1=""
                        change2=""
                    if row%2==0:
                        corner2instructions=str(jtype1[TotalRows-row])+" "+stitch+change1+" / "+str(jtype2[TotalRows-row])+" "+stitch+change2
                    else:
                        corner2instructions=str(jtype2[TotalRows-row])+" "+stitch+change2+" / "+str(jtype1[TotalRows-row])+" "+stitch+change1
            elif corner2=="IL" or corner2=="OL":            
                corner2instructions="" #the 2ch or 2 dec is written when it's corner1
                if corner2=="IL":
                    straight2=laddstraight[row]
                else:
                    straight2=laddstraight[TotalRows-row]
            elif corner2=="IC" or corner2=="OC":
                straight2=caddstraight
#                corner2instructions="" #add the turn if it's corner1

            else:
                print("unknown corner2 type")
    ##########################################
    #straight middle
    ##########################################
            straighttotal=straight1+straightmiddle+straight2
            straightstitches=round(straighttotal)
    ###########################################
    #segment instructions
    ############################################
            if row%2==0:
                if m==len(lengths)-1:#last segment: don't add a final / 
                    if corner2[1]=="C" or corner2[1]=="L":#no corner2
                        instructions[row]=instructions[row]+corner1instructions+" / "+str(straightstitches)+" "+stitch
                    else:
                        instructions[row]=instructions[row]+corner1instructions+" / "+str(straightstitches)+" "+stitch+"  / "+corner2instructions
                else:
                    if corner2[1]=="C" or corner2[1]=="L":#no corner2
                        instructions[row]=instructions[row]+corner1instructions+" / "+str(straightstitches)+" "+stitch+" / "
                    else:
                        instructions[row]=instructions[row]+corner1instructions+" / "+str(straightstitches)+" "+stitch+" / "+corner2instructions+" / "
            else:#add to the LEFT, because we're going right-to-left
                if m==0: #first segment, which is shown last because we're going right-to-left:
                    if corner2[1]=="C" or corner2[1]=="L":
                        instructions[row]=str(straightstitches)+" "+stitch+" / "+corner1instructions+instructions[row]
                    else:
                        instructions[row]=corner2instructions+" / "+str(straightstitches)+" "+stitch+" / "+corner1instructions+instructions[row]
                else:
                    if corner2[1]=="C" or corner2[1]=="L":
                        instructions[row]=str(straightstitches)+" "+stitch+" / "+corner1instructions+instructions[row]+" / "
                    else:
                        instructions[row]=corner2instructions+" / "+str(straightstitches)+" "+stitch+" / "+corner1instructions+instructions[row]+" / "
    return instructions


In [16]:
print(run(["ID",3,"IJ"]))

['7 \\ch~ / 14 \\ch~  / 0 \\ch~ / 4 \\ch~', '5 \\sc~ (+1) / 0 \\sc~ / 14 \\sc~ / 8 \\sc~(+1)', '8 \\sc~ / 14 \\sc~  / 1 \\sc~ (+1, new segment) / 6 \\sc~ (+1)', '6 \\sc~ / 3 \\sc~ (+2) / 14 \\sc~ / 9 \\sc~(+1)', '10 \\sc~(+1) / 14 \\sc~  / 4 \\sc~ (+1) / 7 \\sc~ (+1)', '8 \\sc~ (+1) / 5 \\sc~ (+1) / 14 \\sc~ / 10 \\sc~', '11 \\sc~(+1) / 14 \\sc~  / 7 \\sc~ (+2) / 8 \\sc~']


In [17]:
#print(run(["OJ",3,"OC",3,"OC",2,"IJ"]))
#run this to check that all combinations compile without errors
for lc in ("OJ","OD","IJ","ID"):
    for rc in ("IJ","ID","OJ","OD"):
        for middle1 in ("OC","IC","OL","IL"):
            for middle2 in ("OC","IC","OL","IL"):
                if lc[0]==middle1[0]:
                    n1=3
                else:
                    n1=2
                if middle1[0]==middle2[0]:
                    n2=3
                else:
                    n2=2
                if middle2[0]==rc[0]:
                    n3=3
                else:
                    n3=2
                run([lc,n1,middle1,n2,middle2,n3,rc])
#                print(lc,n1,middle1,n2,middle2,n3,rc)
#                print(run([lc,n1,middle1,n2,middle2,n3,rc]))
#                print()


# Pattern
The pattern needs to be a list of *components*. For standard components, use entries from these options: S, IQ, OQ, IP, OP, IT, OT
(Straight, Inside Quarter turn, Outside Quarter turn, Inside Pint, Outside point, Inner Twist, Outer Twist). The program will compute the correct values for you.

Any other component, including runs, can be given as a list of strings: [instructions for row 0, instructions for row 1,...]. This way you can insert your own custom component wherever you like.

The markers are also a list of strings. There should be one fewer marker than pattern segments.

In [207]:
pattern=[]
displaypattern=[]

print("Enter the components that make up your knot.")
drop=widgets.Dropdown(
    options=["Straight","Inner Quarter","Inner Point","Inner Twist","Outer Quarter", "Outer Point","Outer Twist","Run"],
    value="Straight",
    description='Component:',
    disabled=False,
)

###################################################
run1drop=widgets.Dropdown(
    options=["Inner D-type","Inner J-type","Inner L-type","Inner C-type","Outer D-type", "Outer J-type","Outer L-type","Outer C-type"],
    value="Inner D-type",
    description='First corner:',
    disabled=False,
)
run3drop=widgets.Dropdown(
    options=["Inner D-type","Inner J-type","Inner L-type","Inner C-type","Outer D-type", "Outer J-type","Outer L-type","Outer C-type"],
    value="Inner D-type",
    description='Second corner:',
    disabled=False,
)

run2int=widgets.BoundedIntText(
    value="3",
    min=1,
    description='Length:'
)

runbutton = widgets.Button(description="add run")

def on_runbutton_clicked(b):

    if run1drop.value=="Inner D-type":
        corner1="ID"
    elif run1drop.value=="Inner J-type":
        corner1="IJ"
    elif run1drop.value=="Inner L-type":
        corner1="IL"
    elif run1drop.value=="Inner C-type":
        corner1="IC"
    elif run1drop.value=="Outer D-type":
        corner1="OD"
    elif run1drop.value=="Outer J-type":
        corner1="OJ"
    elif run1drop.value=="Outer L-type":
        corner1="OL"
    elif run1drop.value=="Outer C-type":
        corner1="OC"
        
    if run3drop.value=="Inner D-type":
        corner2="ID"
    elif run3drop.value=="Inner J-type":
        corner2="IJ"
    elif run3drop.value=="Inner L-type":
        corner2="IL"
    elif run3drop.value=="Inner C-type":
        corner2="IC"
    elif run3drop.value=="Outer D-type":
        corner2="OD"
    elif run3drop.value=="Outer J-type":
        corner2="OJ"
    elif run3drop.value=="Outer L-type":
        corner2="OL"
    elif run3drop.value=="Outer C-type":
        corner2="OC"

    displaypattern.append("run("+corner1+str(run2int.value)+corner2+")")
    pattern.append(run([corner1,run2int.value,corner2]))
#    if len(pattern)>5:
#        whichpatterntoshow=pattern
#    else:
#        whichpatterntoshow=displaypattern
    with output:
            print("Pattern:",displaypattern)



###################################################
addbutton = widgets.Button(description="add standard component")

def on_addbutton_clicked(b):
    displaypattern.append(drop.value)
    if drop.value=="Straight":
        pattern.append("S")
    elif drop.value=="Inner Quarter":
        pattern.append("IQ")
    elif drop.value=="Inner Point":
        pattern.append("IP")
    elif drop.value=="Inner Twist":
        pattern.append("IT")
    elif drop.value=="Outer Quarter":
        pattern.append("OQ")
    elif drop.value=="Outer Point":
        pattern.append("OP")
    elif drop.value=="Outer Twist":
        pattern.append("OT")
         
#    if len(pattern)>5:
#        whichpatterntoshow=pattern
#    else:
#        whichpatterntoshow=displaypattern
    with output:
        print("Pattern:",displaypattern)



###################################################

deletebutton = widgets.Button(description="delete last")
output = widgets.Output()

def on_deletebutton_clicked(b):
    if len(pattern)!=0:
        del pattern[-1]
        del displaypattern[-1]
    with output:
        print("Pattern:",displaypattern)



###################################################
#display(drop)
#display(addbutton)
#display(run1drop,run2int,run3drop)
#display(runbutton)
#display(deletebutton,output)
addbutton.on_click(on_addbutton_clicked)
runbutton.on_click(on_runbutton_clicked)
deletebutton.on_click(on_deletebutton_clicked)


left_box = widgets.VBox([drop, addbutton])
right_box = widgets.VBox([run1drop,run2int,run3drop,runbutton])
widgets.HBox([left_box, right_box,output])

#items = [drop,run1drop,addbutton,run2int,run3drop,addbutton,runbutton]

#widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 300px)"))



Enter the components that make up your knot.


In [196]:
markers=[]

print("Enter the marker labels:")

markernumber=widgets.Text(
    value='1T',
    description='Label:',
    disabled=False
)
###################################################
addmarkerbutton = widgets.Button(description="add label")

output = widgets.Output()

def on_addmarkerbutton_clicked(b):
    if len(markers)<len(pattern)-1:
        markers.append(markernumber.value)
        with output:
            print("Markers:",markers)
    else:
        print("The number of markers should be smaller than the number of components in the pattern")
addmarkerbutton.on_click(on_addmarkerbutton_clicked)

###################################################

deletemarkerbutton = widgets.Button(description="delete last")
output = widgets.Output()

def on_deletemarkerbutton_clicked(b):
    if len(markers)!=0:
        del markers[-1]
    with output:
        print("Markers:",markers)

deletemarkerbutton.on_click(on_deletemarkerbutton_clicked)


###################################################

widgets.HBox([markernumber,addmarkerbutton,deletemarkerbutton,output])


Enter the marker labels:


The number of markers should be smaller than the number of components in the pattern
The number of markers should be smaller than the number of components in the pattern
The number of markers should be smaller than the number of components in the pattern
The number of markers should be smaller than the number of components in the pattern
The number of markers should be smaller than the number of components in the pattern
The number of markers should be smaller than the number of components in the pattern


In [194]:


#pattern=["S","IQ","OQ","IP","OP","IT","OT","OR4","OR3"]
#markers=["1T","2B","3T","4B","5T","6B","7T","8B","9T","10B"]

#a=["row0","row1","row2","row3","row4","ow5","row6","row7","row8","row9}"]
#a=run(["ID",5,"IC",5,"IJ"])
#pattern=[a,"S","S","IQ","S","S","S","OQ","S","S","S","IQ","S","S","S","OQ","S","S","S","IQ","S","S","S","OQ","S","S","S","IQ","S"]
#markers=["1T","2B","2T","3B","4T","5B","5T","6B","3T","7B","7T","8B","9T","8B","8T","9B","8T","10B","10T","5B","6T","3B","3T","4B","5T","11B","11T"]

#RO=run(["OD",5,"OD"])

#RI=run(["ID",5,"ID"])

#pattern=["S","S","S",RO,"S","S","S","S",RI,"S","S","S","S",RO,"S","S"]#B

#markers=["20T","5B","11T","10B","5T","21B","8T","7B","10T","8B","21T","3B","7T","2B","3T"]

#pattern=["S","S","IQ","S","S","S","OQ","S","S","S","IQ","S","S","S","OQ","S","S","S","IQ","S","S","S","OQ","S","S","S","IQ","S"]

#markers=["1T","2B","2T","3B","4T","5B","5T","6B","3T","7B","7T","8B","9T","8B","8T","9B","8T","10B","10T","5B","6T","3B","3T","4B","5T","11B","11T"]

##C
#pattern=["S","S",run(["ID",3,"IJ"]),"S"]
#markers=["31T","11B","32T"]

##D
pattern=["S","S",run(["IJ",3,"ID"]),"S"]
markers=["34T","33B","2T"]

#pattern=["S","S","IQ","S","S","S","OQ","S","S","S","IQ","S","S","S","OQ","S","S","S","IQ","S","S","S","OQ","S","S","S","IQ","S"]
#markers=["1T","2B","2T","3B","4T","5B","5T","6B","3T","7B","7T","8B","9T","8B","8T","9B","8T","10B","10T","5B","6T","3B","3T","4B","5T","11B","11T"]

###D
#RO=run(["OD",5,"OD"])
#RI=run(["ID",5,"ID"])
#pattern=["S","S","S",RO,"S","S","S","S",RI,"S","S","S","S",RO,"S","S"]
#markers=["20T","5B","11T","10B","5T","21B","8T","7B","10T","8B","21T","3B","7T","2B","3T"]


The output code is formatted to be put inside a description environment in latex. 

Custom macro \m{} prints labelled running stitch markers. This seems helpful for the foundation chain, but just clutters the pattern after, so it's only printed once.

Macros \sc and \ch for single crochet and chains. I like to supporess them, so I define them to do nothing; easy to change back to abbreviations, or to 

# Generate Pattern

In [11]:
# Output ormatted to be put into LaTeX, with macros \m{} for markers and stitches \sc, \ch
# Need to define pattern and markers as lists of strings

##if the markers are too short. In particular, if you don't want to specify marker labels, just make it an empty string.
if len(markers)==0:
    for i in range(len(pattern)):
        markers.append("")
elif len(markers)<len(pattern)-1:#in this case there are some markers but not enough. Add question marks so it's clear these aren't intentional blanks
    for i in range(len(pattern)-len(markers)-1):
        markers.append("?")


#pattern printing starts here
print("\\begin{description}")
for r in range(TotalRows+1):
        #print row number
        #foundation row uses chains, other rows use sc
        if r==0:
            print("\item[Foundation Chain:] (purple)")
            stitch="\ch"
            countallstitches=0
        else:
            if r==1 or r==6:
                color="purple"
            elif r==2 or r==5:
                color="red"
            else:
                color="orange"
            print("\item[Row "+str(r)+":] (",color,")")
            stitch="\sc~"
        if (r%2)==0:#even rows and odd rows work the pattern in opposite directions
            marktrack = 0 #markers[0] starts the row
            markincrement=1
        else:
            marktrack = len(pattern)-2 #markers[n] starts the row
            markincrement=-1
        #even rows: pattern[0], pattern[1]. 
        #odd rows: pattern[n],pattern[n-1]. 
        for SegmentNumber in range(len(pattern)):
            if (r % 2) == 0:
                seg = pattern[SegmentNumber]
            else:
                ReverseSegmentNumber=len(pattern)-SegmentNumber-1
                seg = pattern[ReverseSegmentNumber]
        #print instructions: number of sc and number of increase, if not straight
            #straight
            if seg == "S":
                if r==0: #markers are numbered only when r=0
                    if SegmentNumber==len(pattern)-1: #no marker after the last segment
                        end=""
                    else:
                        end="\m{"+str(markers[marktrack])+"}"
                    print (dtd,stitch,end)
                else:
                    if SegmentNumber==len(pattern)-1:
                        end=""
                    else:
                        end="/"
                    print (dtd,stitch,end)
                marktrack = marktrack + markincrement
            #quarter turns
            elif seg=="IQ" or seg=="OQ":
                if seg=="IQ":
                    SegmentSC=QuarterTurn[r]
                    if r!=0:
                        SegmentChange="+"
                        SegmentDelta=QuarterTurn[r]-QuarterTurn[r-1]
                        if SegmentDelta==0:
                            change=""
                        else:
                            change="("+SegmentChange+str(SegmentDelta)+")"
                elif seg=="OQ":
                    SegmentSC=QuarterTurn[TotalRows-r]
                    if r!=0:
                        SegmentChange="-"
                        SegmentDelta=QuarterTurn[TotalRows-(r-1)]-QuarterTurn[TotalRows-r]
                        if SegmentDelta==0:
                            change=""
                        else:
                            change="("+SegmentChange+str(SegmentDelta)+")"
                if r==0:
                    if SegmentNumber==len(pattern)-1:
                        end=""
                    else:
                        end="\m{"+str(markers[marktrack])+"}"
                    print(QTstraightsts,stitch,"/",SegmentSC,stitch,"/",QTstraightsts,stitch,end)
                    marktrack = marktrack + markincrement
                else:
                    if SegmentNumber==len(pattern)-1:
                        end=""
                    else:
                        end="/"
                    print(QTstraightsts,stitch,"/",SegmentSC,stitch+change,"/",QTstraightsts,stitch,end)
                    marktrack = marktrack + markincrement
            #points
            elif seg=="IP" or seg=="OP":
                if seg=="IP":
                    SegmentSC=PointArc[r]
                    CornerStyle="2 ch,"
                    if r!=0:
                        #arc part
                        SegmentChange="+"
                        SegmentDelta=PointArc[r]-PointArc[r-1]
                        if SegmentDelta==0:
                            change=""
                        else:
                            change="("+SegmentChange+str(SegmentDelta)+")"
                        #straight part
                        #PointCorner gives the measure of a side; one stitch will be a chain, the others sc
                        StraightPart=PointCorner[r]-1                         

                    else: #foundation chain row: both sides consist of chains
                        StraightPart=2*PointCorner[r]

                elif seg=="OP":
                    SegmentSC=PointArc[TotalRows-r]
                    CornerStyle="2 dec sts,"
                    if r!=0:
                        #arc part
                        SegmentChange="-"
                        SegmentDelta=PointArc[TotalRows-(r-1)]-PointArc[TotalRows-r]
                        if SegmentDelta==0:
                            change=""
                        else:
                            change="("+SegmentChange+str(SegmentDelta)+")"
                        #straight part
                        #PointCorner gives the measure of a side; one stitch will be a chain, the others sc
                        StraightPart=PointCorner[TotalRows-r]-1 
                    else:#foundation chain row: everything is a chain
                        StraightPart=2*PointCorner[TotalRows-r]
                if r==0:
                    if SegmentNumber==len(pattern)-1:
                        end=""
                    else:
                        end="\m{"+str(markers[marktrack])+"}"
                    print(SegmentSC,"\ch / ",StraightPart,"\ch / ",SegmentSC,end)
                    marktrack = marktrack + markincrement
                else:
                    if SegmentNumber==len(pattern)-1:
                        end=""
                    else:
                        end="/"
                    print(SegmentSC,"\sc~"+change,"/ ",StraightPart,"sc,",CornerStyle,StraightPart,"sc / ",SegmentSC,"\sc~"+change,end)
                    marktrack = marktrack + markincrement
            elif seg=="IT" or seg=="OT":
                if seg=="IT":
                    SegmentSCOuter=PointArc[r]
                    SegmentSCInner=TwistTop[r]
                    if r!=0:
                        SegmentChange="+"
                        SegmentDeltaOuter=PointArc[r]-PointArc[r-1]
                        SegmentDeltaInner=TwistTop[r]-TwistTop[r-1]
                        if SegmentDeltaOuter==0:
                            changeOuter=""
                        else:
                            changeOuter="("+SegmentChange+str(SegmentDeltaOuter)+")"
                        if SegmentDeltaInner==0:
                            changeInner=""
                        else:
                            changeInner="("+SegmentChange+str(SegmentDeltaInner)+")"
                elif seg=="OT":
                    SegmentSCOuter=PointArc[TotalRows-r]
                    SegmentSCInner=TwistTop[TotalRows-r]
                    if r!=0:
                        SegmentChange="-"
                        SegmentDeltaOuter=PointArc[TotalRows-(r-1)]-PointArc[TotalRows-r]
                        SegmentDeltaInner=TwistTop[TotalRows-(r-1)]-TwistTop[TotalRows-r]
                        if SegmentDeltaOuter==0:
                            changeOuter=""
                        else:
                            changeOuter="("+SegmentChange+str(SegmentDeltaOuter)+")"
                        if SegmentDeltaInner==0:
                            changeInner=""
                        else:
                            changeInner="("+SegmentChange+str(SegmentDeltaInner)+")"

                if r==0:
                    if SegmentNumber==len(pattern)-1:
                        end=""
                    else:
                        end="\m{"+str(markers[marktrack])+"}"
                    print(SegmentSCOuter,stitch,"/",SegmentSCInner,stitch,"/",SegmentSCOuter,end)
                    marktrack = marktrack + markincrement
                else:
                    if SegmentNumber==len(pattern)-1:
                        end=""
                    else:
                        end="/"
                    print(SegmentSCOuter,stitch,changeOuter," / ",SegmentSCInner,stitch,changeInner," / ",SegmentSCOuter,"\sc~"+changeOuter,end)
                    marktrack = marktrack + markincrement
            else:
                if SegmentNumber==len(pattern)-1:
                    end=""
                else:
                    if r==0:
                        end="\m{"+str(markers[marktrack])+"}"
                    else:
                        end="/"
                print(seg[r],end)
                marktrack = marktrack + markincrement

print("\\end{description}")

\begin{description}
\item[Foundation Chain:] (purple)
10 \ch \m{34T}
10 \ch \m{33B}
4 \ch~ / 0 \ch~ / 14 \ch~  / 7 \ch~  \m{2T}
10 \ch 
\item[Row 1:] ( purple )
10 \sc~ /
8 \sc~ (+1) / 14 \sc~ / 0 \sc~ / 5 \sc~ (+1) /
10 \sc~ /
10 \sc~ 
\item[Row 2:] ( red )
10 \sc~ /
10 \sc~ /
6 \sc~ (+1) / 1 \sc~ (+1, new segment) / 14 \sc~  / 8 \sc~  /
10 \sc~ 
\item[Row 3:] ( orange )
10 \sc~ /
9 \sc~ (+1) / 14 \sc~ / 3 \sc~ (+2) / 6 \sc~ /
10 \sc~ /
10 \sc~ 
\item[Row 4:] ( orange )
10 \sc~ /
10 \sc~ /
7 \sc~ (+1) / 4 \sc~ (+1) / 14 \sc~  / 10 \sc~ (+1) /
10 \sc~ 
\item[Row 5:] ( red )
10 \sc~ /
10 \sc~  / 14 \sc~ / 5 \sc~ (+1) / 8 \sc~ (+1) /
10 \sc~ /
10 \sc~ 
\item[Row 6:] ( purple )
10 \sc~ /
10 \sc~ /
8 \sc~ / 7 \sc~ (+2) / 14 \sc~  / 11 \sc~ (+1) /
10 \sc~ 
\end{description}
